# 連想配列と連結リスト


## キーポイント

*


----

## 1 連想配列

----

### 1.1 find関数の比較回数

「文章に特定の単語が含まれているか調べるプログラム」を作ることを考えます。<br>
思いつく方法として、次のようなものがあります。

1. `vector<string>`に単語単位で文章を読み込む
2. `find`関数で単語を探す

この方針で作成したプログラムを次に示します。

In [ ]:
# @title コード
%%writefile find_word_using_find.cpp
#include <iostream>
#include <fstream>
#include <vector>
#include <string>
#include <algorithm>
using namespace std;

int main() {
  vector<string> v;
  v.reserve(1000);

  fstream ifs("sample.txt");
  for (;;) {
    string s;
    ifs >> s;
    if (ifs.eof()) {
      break;
    }
    v.push_back(s);
  }

  string x;
  cin >> x;

  auto i = ranges::find(v, x);

  if (i != v.end()) {
    cout << x << " is found." << endl;
  } else {
    cout << x << " is not found." << endl;
  }
}

In [ ]:
# @title 実行テスト
!echo "The names John Doe for males, Jane Doe or Jane Roe for females, or Jonnie Doe and Janie Doe for children, or just Doe non-gender-specifically are used as placeholder names for a party whose true identity is unknown or must be withheld in a legal action, case, or discussion. The names are also used to refer to acorpse or hospital patient whose identity is unknown. This practice is widely used in the United States and Canada, but is rarely used in other English-speaking countries including the United Kingdom itself, from where the use of John Doe in a legal context originates. The names Joe Bloggs or John Smith are used in the UK instead, as well as in Australia and New Zealand.">sample.txt
!echo "John Doe is sometimes used to refer to a typical male in other contexts as well, in a similar manner to John Q. Public, known in Great Britain as Joe Public, John Smith or Joe Bloggs. For example, the first name listed on a form is often John Doe, along with a fictional address or other fictional information to provide an example of how to fill in the form. The name is also used frequently in popular culture, for example in the Frank Capra film Meet John Doe. John Doe was also the name of a 2002 American television series.">>sample.txt
!echo "Similarly, a child or baby whose identity is unknown may be referred to as Baby Doe. A notorious murder case in Kansas City, Missouri, referred to the baby victim as Precious Doe. Other unidentified female murder victims are Cali Doe and Princess Doe. Additional persons may be called James Doe, Judy Doe, etc. However, to avoid possible confusion, if two anonymous or unknown parties are cited in a specific case or action, the surnames Doe and Roe may be used simultaneously; for example, John Doe v. Jane Roe. If several anonymous parties are referenced, they may simply be labelled John Doe #1, John Doe #2, etc. (the U.S. Operation Delego cited 21 (numbered) John Doe s) or labelled with other variants of Doe / Roe / Poe / etc. Other early alternatives such as John Stiles and Richard Miles are now rarely used, and Mary Major has been used in some American federal cases." >> sample.txt
!g++ -std=c++20 -O2 -Wall -Wextra -o find_word_using_find find_word_using_find.cpp && echo "木の下をクリックして検索したい単語を入力" && ./find_word_using_find

`find`関数による検索は、基本的にはfor文と同じで、範囲の先頭からひとつずつ順番に、条件に合うかどうかを調べます。<br>
つまり、`find`関数を使った次のプログラムは、

```cpp
auto i = ranges::find(v, x);
```

for文を使った次のプログラムと同じです。

```cpp
auto i = v.begin();
for (; i != v.end(); i++) {
  if (*i == x) {
    break;
  }
}
```

`find`関数とfor文に共通の問題は「データ量に比例して検索に時間がかかるようになる」ことです。<br>
データがランダムな位置で見つかると過程すると、データを見つけるまでに平均して`データ数 / 2`回の比較が行われます。<br>
例えば、単語の数が10万語の文章が入力された場合、平均比較回数は5万回です。

2025年現在のコンピューターであれば、5万回程度の比較は1秒とかからず完了するでしょう。ですが、ゲームのように1/60秒以内に様々な処理を行わなくてはならないアプリや、多くの人から閲覧されて大量のリクエストを受けるWebサイトでは、もっと短い時間で処理を終了させなくてはなりません。


### 1.2 unordered_set(アンオーダード・セット)

検索を高速化する方法として、適切なコンテナを使うことが挙げられます。<br>
例えば、「連想配列(れんそうはいれつ)」という種類のコンテナを使うと、1回比較するだけでデータを見つけられます。

連想配列は「どんなデータでも添字にできる配列」です。添字に使うデータは、「ハッシュ化」という特別な加工をして、連想配列に格納されます。詳細は省きますが、「ハッシュ化と格納方法の工夫により、データの検索を非常に高速に行える」のが、連想配列の利点です。

C++の連想配列は全部で8種類ありますが、主に使われるのは次の2つです。

* `unordered_set`(アンオーダード・セット)
* `unordered_map`(アンオーダード・マップ)

`vector`と直接対応するのは`unordered_set`です。前のプログラムを`unordered_set`を使って置き換えると、次のようになります。

**コード**

```cpp
#include <iostream>
#include <unordered_set>
#include <string>
using namespace std;

int main() {
  unordered_set<string> v;
  v.reserve(10'000);

  while (cin) {
    string s;
    cin >> s;
    v.insert(s);
  }

  string x;
  cin >> x;

  auto i = v.find(x);

  if (i != v.end()) {
    cout << "Yes" << endl;
  } else {
    cout << "No" << endl;
  }
}
```

`unordered_set`にデータを追加するには、`insert`(インサート)関数を使います。<br>
データの検索には、`unordered_set`自身の`find`関数を使います(汎用アルゴリズムの`find`は使いません)。

`unordered_set`の`find`関数を使うと、どれほどデータ数が多くなっても、基本的には1回の比較で目的のデータを見つけられます。




### 1.2 unordered_map


**コード**

```cpp
#include <iostream>
#include <unordered_map>
#include <string>
using namespace std;

int main() {
  unordered_map<char, string> v = { {'y', "andy"}, {'e', "barbie"}, {'d', "conrad"}, {'s', "davis"}, {'i', "elli"}, {'n', "finn"} };
  cout << v['s'] << endl;
}
```

**実行結果**

```txt
davis
```

`unordered_map`では、「添え字」と「データ」をペアにして追加します。<br>
上記のプログラムの場合、最初のペアは`{'y', "andy"}`の部分で、添え字は`'y'`、データは`"andy"`となります。

目的のデータがコンテナに格納されている場合、上記のプログラムのように、キーを使ってデータを読み書きできます。

>**【キー(鍵)】**<br>
>連想配列では、添え字のことを「キー(鍵)」と呼びます(添え字が「数字」とは限らないため)。<br>
>今後は、キーと書かれていたら「連想配列の添え字」を意味するものとします。


#### pair構造体

連想配列は、キーとデータをペアにして格納します。<br>
標準ライブラリにはペアを作るために、その名も`pair`(ペア)という構造体があります。

`pair`構造体は次のようなものです。

```cpp
struct pair<１個目のデータの型, ２個目のデータの型> {
  １個目のデータの型  first;  // １個目のデータ
  ２個目のデータの型  second; // ２個目のデータ
};
```

例えば、`unordered_map<char, string>`型が使用するペアは、次のようになります。

```cpp
struct pair<char, string> {
  char   first;
  string second;
};
```


#### 連想配列のデータの有無を調べる

連想配列は何でもキーにできるため、「キーに対応するデータがない」場合のことも考えなくてはなりません。<br>
例えば、上記のプログラムでキーを`a`に変えたとします。しかし、連想配列の中にキーが`a`のデータはありません。

あるキーに対応するデータが存在することを調べるには、`find`(ファインド)メンバ関数を使います。

```cpp
bool find(キー)
```

**コード**

```cpp
#include <iostream>
#include <unordered_map>
#include <string>
using namespace std;

int main() {
  unordered_map<char, string> v = { {'y', "andy"}, {'e', "barbie"}, {'d', "conrad"}, {'s', "davis"}, {'i', "elli"}, {'n', "finn"} };
  auto i = v.find('a');
  if (i == v.end()) {
    cout << "aで終わる名前はありません" << endl;
  } else {
    cout << i->second << endl;
  }
}
```

**実行結果**

```txt
aで終わる名前はありません
```

`find`メンバ関数の戻り値はイテレータです。イテレータは「キーとデータのペア」の位置をあらわします。<br>
そのため、`second`変数によってデータを読み書きできます。


#### 連想配列にデータを追加する

`unordered_map`にデータを追加するには`try_emplace`(トライ・エンプレイス)メンバ関数を使います。

```cpp
pair<iterator, bool> try_emplace(キー, データ)
```

このメンバ関数の戻り値は「追加したデータの位置をあらわすイテレータ」と、「追加の結果をあらわす真偽値」です。

**コード**

```cpp
#include <iostream>
#include <unordered_map>
#include <string>
using namespace std;

int main() {
  unordered_map<char, string> v = { {'y', "andy"}, {'e', "barbie"}, {'d', "conrad"}, {'s', "davis"}, {'i', "elli"}, {'n', "finn"} };

  // キーがaの要素はないので成功する
  auto a = v.try_emplace('a', "gaia");
  if (a.second) {
    cout << a.first->second << "を追加" << endl;
  } else {
    cout << "gaiaの追加に失敗" << endl;
  }

  // キーがaの要素はすでにあるので失敗する
  auto b = v.try_emplace('a', "hanna");
  if (b.second) {
    cout << a.first->second << "を追加" << endl;
  } else {
    cout << "hannaの追加に失敗" << endl;
  }

  // キーがaの要素を検索
  auto i = v.find('a');
  if (i == v.end()) {
    cout << "aで終わる名前はありません" << endl;
  } else {
    cout << i->second << endl;
  }
}
```

**実行結果**

```txt
gaiaを追加
hannaの追加に失敗
gaia
```


#### 連想配列からデータを削除する

`unordered_map`からデータを削除するには`erase`(イレーズ)メンバ関数を使います。<br>
なお、設計上の理由から、汎用アルゴリズムの`erase`は連想配列に対応していないので使えません。<br>
注意してください(ただし、`erase_if`汎用アルゴリズムは使えます)。

```cpp
削除した数 erase(削除するキー)
```

**コード**

```cpp
#include <iostream>
#include <unordered_map>
#include <string>
using namespace std;

int main() {
  unordered_map<char, string> v = { {'y', "andy"}, {'e', "barbie"}, {'d', "conrad"}, {'s', "davis"}, {'i', "elli"}, {'n', "finn"} };

  v.erase('s');

  auto i = v.find('s');
  if (i == v.end()) {
    cout << "sで終わる名前はありません" << endl;
  } else {
    cout << i->second << endl;
  }
}
```

**実行結果**

```txt
sで終わる名前はありません
```

>**【連想配列用のerase汎用アルゴリズムがない理由】**<br>
>たいていのコンテナの`erase`メンバ関数と`erase`汎用アルゴリズムは、「`==`による比較」を行います。しかし、連想配列の`erase`メンバ関数は「`<`による比較」を行います。この違いから、連想配列の`erase`メンバ関数のほうが、`erase`汎用アルゴリズムよりも高速に動作します。つまり、連想配列用の`erase`汎用アルゴリズムを作っても、使う価値がないのです。
>
>対して、`erase_if`汎用アルゴリズムは「比較方法を任意に決められる」のが特徴です。これは、メンバ関数として作っても、通常の関数として作っても、実行速度が変わらないことを意味します。そのため、汎用アルゴリズムだけが用意されているのです。


#### 空きの多い配列として使う

$100m^2$ の平面空間に、100体の敵がいて、敵同士が重ならないようにしたいとします。

敵同士の衝突判定は、1体につき残り99体の敵との衝突判定を行うことで実現できます。この方法では、9900回の衝突判定を行う必要があります。

しかし、「衝突するほど近い距離にいる敵の数」は、多くても10体くらいでしょう。<br>
ということは、「衝突するほど近い敵」を判別できれば、衝突判定の回数を、1000回程度まで減らせるはずです。

このような場合、例えば $100m^2$ 空間を $5m^2$ 単位に分割して、20x20マスの配列を作ります。すべての敵を「位置に対応するマス」に登録します。<br>
このデータ構造では、同じマスにいる敵は「衝突するほど近い敵」と判定できます。

**配列の例(4x4マスに8体の敵がいる)**

```txt
   0   5  10  15  20
 0┌─┬─┬─┬─┐    v[0]  = { 敵1, 敵2 }
  │ 0│ 1│ 2│ 3│    v[1]  = {}
 5├─┼─┼─┼─┤    v[2]  = { 敵6 }
  │ 4│ 5│ 6│ 7│    v[3]  = {}
10├─┼─┼─┼─┤    v[4]  = { 敵4, 敵8 }
  │ 8│ 9│10│11│    v[5]  = {}
15├─┼─┼─┼─┤    v[6]  = {}
  │12│13│14│15│    v[7]  = {}
20└─┴─┴─┴─┘    v[8]  = {}
                        v[9]  = {}
                        v[10] = { 敵3 }
                        v[11] = {}
                        v[12] = {}
                        v[13] = {}
                        v[14] = {}
                        v[15] = { 敵5, 敵7 }
```

**コード**

```cpp
#include <iostream>
#include <vector>
using namespace std;

struct Enemy {
  bool HitCheck(const Enemy& other) {
    return abs(x - other.x) <= 1 && abs(y - other.y) <= 1;
  }

  int ToIndex() { return (y / 5) * 4 + (x / 5); }

  int x, y;
};

int main() {
  // 敵の配列
  vector<Enemy> enemies = { { 2, 3}, { 4, 1}, {13,13}, { 3, 7},
                            {16,17}, {11, 2}, {18,16}, { 3, 8} };
  // 敵をマス目に登録
  vector<vector<Enemy*>> v(16);
  for (int i = 0; i < (int)enemies.size(); i++) {
    int index = enemies[i].ToIndex();
    v[index] = &enemies[i];
  }

  // マス目の敵の衝突判定
  for (int i = 0; i < (int)v.size(); i++) {
    for (int a = 0; a < (int)v[i].size() - 1; a++) {
      Enemy* pa = v[i][a];
      for (int b = a + 1; b < (int)v[i].size(); b++) {
        Enemy* pb = v[i][b];
        if (pa->HitCheck(*pb)) {
          cout << pa->x << ',' << pa->y << ' ' << pb->x << ',' << pb->y << endl;
        }
      } // for b
    } // for a
  } // for i
}
```

**出力結果**

```txt
3,7 3,8
```

>この例では省略していますが、本来、面積を持つ物体の衝突判定では、周囲のマスの敵とも衝突判定を行う必要があります。

２次元配列を使う場合の問題点は、「敵のいないマスのメモリが無駄になる」ということです。<br>
例えば、上の図のように16マス中5マスしか敵がいない場合、11マスは全く使われません。

このような、「空きが多い配列」を効率的に表現するには、連想配列を使います。<br>
例えば、敵のいるマスの番号をキーとする連想配列を作ります。そして、敵のいるマスだけを連想配列に追加します。

**連想配列の例(4x4マスに8体の敵がいる)**

```txt
   0   5  10  15  20
 0┌─┬─┬─┬─┐    m[0]  = { 敵1, 敵2 }
  │ 0│ 1│ 2│ 3│    m[2]  = { 敵6 }
 5├─┼─┼─┼─┤    m[4]  = { 敵4, 敵8 }
  │ 4│ 5│ 6│ 7│    m[10] = { 敵3 }
10├─┼─┼─┼─┤    m[15] = { 敵5, 敵7 }
  │ 8│ 9│10│11│
15├─┼─┼─┼─┤
  │12│13│14│15│
20└─┴─┴─┴─┘
```

**コード**

```cpp
#include <iostream>
#include <vector>
#include <unordered_map>
using namespace std;

struct Enemy {
  bool HitCheck(const Enemy& other) {
    return abs(x - other.x) <= 1 && abs(y - other.y) <= 1;
  }

  int ToIndex() { return (y / 5) * 4 + (x / 5); }

  int x, y;
};

int main() {
  // 敵の配列
  vector<Enemy> enemies = { { 2, 3}, { 4, 1}, {13,13}, { 3, 7},
                            {16,17}, {11, 2}, {18,16}, { 3, 8} };
  // 敵をマス目に登録
  unordered_map<int, vector<Enemy*>> m;
  for (int i = 0; i < (int)enemies.size(); i++) {
    int index = enemies[i].ToIndex();
    m[index].push_back(&enemies[i]);
  }

  // マス目の敵の衝突判定
  for (auto i = m.begin(); i != m.end(); i++) {
    for (int a = 0; a < (int)i->second.size() - 1; a++) {
      Enemy* pa = i->second[a];
      for (int b = a + 1; b < (int)i->second.size(); b++) {
        Enemy* pb = i->second[b];
        if (pa->HitCheck(*pb)) {
          cout << pa->x << ',' << pa->y << ' ' << pb->x << ',' << pb->y << endl;
        }
      } // for b
    } // for a
  } // for i
}
```

**出力結果**

```txt
3,7 3,8
```

連想配列に追加されるマスは0, 2, 4, 10, 15の5つだけなので、敵のいないマスのために無駄なメモリを使わなくて済みます。

>**【ひとつのキーに複数のデータを割り当てる他の方法】**<br>
>連想配列の例では、複数のデータを割り当てるために配列を使いました。<br>
>他の方法として、`unordered_multimap`(アンオーダード・マルチマップ)というクラスを使うこともできます。<br>
>名前に「マルチ」とあるように、アンオーダード・マルチマップは、同じキーに対して複数のデータを割り当てられます。

>**【キーとデータが同じ場合】**<br>
>キーとデータが等しい、つまりキーだけで十分な場合、`unordered_set`(アンオーダード・セット)クラスを使います。<br>
>アンオーダード・セットはキーしか持たない連想配列です。



### 2.5 連結リスト

「連結(れんけつ)リスト」は、次の特徴を持つコンテナです。

* コンテナ内の要素が物理的には別々の場所にある
* それぞれの要素は、前後の要素とつながっている

C++では、連結リストを`list`クラスとして定義しています。<br>
要素の追加や削除のような基本的な使いかたは、`vector`と同じです。


#### 連結リストのデータ構造

さて、`list`の各要素は、以下のような要素単位のメモが、メモ帳のあちこちに散らばっているのだと説明しました。

```txt
前の要素の位置: ◯ページ△行目
次の要素の位置: □ページ✕行目
データ: 1
```

仮に、データの型を`T`(ティー)とすると、このメモは次のような構造体としてあらわせます。

```cpp
struct ListData {
  ListData* prev; // 前の要素の位置
  ListData* next; // 次の要素の位置
  T data;         // データ
};
```

実際の`list`型の要素とは細部が異なりますが、おおよそはこの形をしていると考えてください。<br>
この種類のデータ構造が「連結リスト」と呼ばれるのは、各データが互いに連結されていることに由来します。


#### 連結リストの利点

連結リストの利点は、以下の操作が`vector`よりはるかに高速に行えることにあります。

* リストの好きな位置にデータを追加する
* リストの好きな位置のデータを削除する

例えば、リストの先頭にデータを追加することを考えてみましょう。

* `vector`の場合：<br>
  `vector`配列は、先頭から末尾までデータがぎっしりと詰まっています。<br>
  先頭にデータを追加するには、すべてのデータをひとつずつ後ろに移動して、先頭に空きを作らなくてはなりません。<br>
  もしもデータが何千、何万と格納されている場合、それらのデータを移動するにはかなりの時間がかかるでしょう。
* `list`の場合：<br>
  `list`は「先頭の要素の位置」と「末尾の要素の位置」の2つポインタ変数を持っています。<br>
  先頭にデータを追加するには、まず「先頭の要素の位置」と、先頭の要素の「前の要素の位置」に、新しい要素の位置を代入します。<br>
  次に新しい要素の「次の要素の位置」に、先ほどまで先頭だった要素の位置を代入します。

この例は先頭への追加ですが、どの位置へ追加する場合でもやることは同じです。大量の要素を移動させる必要はありません。

同じことが、要素の削除にも当てはまります。

* `vector`の場合：<br>
  削除したい要素より後ろにあるすべてのデータを、ひとつずつ前に移動させる必要があります。
* `list`の場合：<br>
  削除したい要素の前後の要素を、互いに連結し直すだけです。

さらに、`list`は「他の`list`から要素を移動」したり、「2つのリストをまとめる」ような操作も非常に高速です。<br>
要素をつなぎ直すだけで済むからです。

**先頭への追加速度の比較**

次のプログラムは、`vector`と`list`について、**先頭**にデータを追加するのにかかる時間を計測し、ミリ秒単位で表示します。<br>
データを追加する前の要素数は`data_count`定数によって変更できます。<br>
実際にどれくらいの時間がかかるのか、追加前の要素数を変えながら、何度か実行してみてください。


In [ ]:
%%writefile list_vs_vector.cpp
#include <iostream>
#include <chrono>
#include <vector>
#include <list>
using namespace std;

int main() {
  // 初期データ数
  //const int data_count = 100;
  //const int data_count = 1'000;
  const int data_count = 10'000;
  //const int data_count = 100'000;
  //const int data_count = 1'000'000;

  cout << "data_count: " << data_count << endl;

  vector<int> n(data_count);
  auto s = chrono::system_clock::now();
  n.insert(n.begin(), 1);
  cout << "vector::insert: " << chrono::duration_cast<chrono::microseconds>(chrono::system_clock::now() - s).count() << "ms" << endl;

  list<int> m(data_count);
  s = chrono::system_clock::now();
  m.insert(m.begin(), 1);
  cout << "list::insert:" << chrono::duration_cast<chrono::microseconds>(chrono::system_clock::now() - s).count() << "ms" << endl;
}

In [ ]:
!g++ list_vs_vector.cpp -o list_vs_vector && ./list_vs_vector

#### 連結リストの欠点

連結リストの欠点は次のとおりです。

* 末尾にデータを追加する処理が`vector`より遅い
* イテレータの種類が「双方向イテレータ」なので、ひとつずつしか位置を移動できない

連結リストの構造は、`vector`のような配列と比べて複雑です。<br>
そのため、複数のデータを格納して、それらを読み書きする、という基本的な用途だけを考えると、`vector`よりも性能が低くなります。

例えば、データを末尾に追加する処理は`vector`のほうが高速です。<br>
このため、末尾にデータを大量に追加することは、`list`が苦手とする操作です。

また、イテレータの種類が「双方向イテレータ」なので、「位置をn個移動する」という操作ができません。<br>
これは「添え字を使えない原因」でもあります。

**末尾への追加速度の比較**

次のプログラムは、`vector`と`list`について、**末尾**にデータを追加するのにかかる時間を計測し、ミリ秒単位で表示します。<br>
追加するデータ数は`data_count`定数によって変更できます。<br>
実際にどれくらいの時間がかかるのか、追加するデータ数を変えながら、何度か実行してみてください。




In [ ]:
%%writefile list_vs_vector_2.cpp
#include <iostream>
#include <chrono>
#include <vector>
#include <list>
using namespace std;

int main() {
  // 追加するデータ数
  //const int data_count = 100;
  const int data_count = 1'000;
  //const int data_count = 10'000;
  //const int data_count = 100'000;
  //const int data_count = 1'000'000;

  cout << "data_count: " << data_count << endl;

  vector<int> n;
  //n.reserve(data_count); // この行を有効にすると、vectorがさらに速くなる
  auto s = chrono::system_clock::now();
  for (int i = 0; i < data_count; i++) {
    n.push_back(i);
  }
  cout << "vector::push_back: " << chrono::duration_cast<chrono::microseconds>(chrono::system_clock::now() - s).count() << "ms" << endl;

  list<int> m;
  s = chrono::system_clock::now();
  for (int i = 0; i < data_count; i++) {
    m.push_back(i);
  }
  cout << "list::push_back:" << chrono::duration_cast<chrono::microseconds>(chrono::system_clock::now() - s).count() << "ms" << endl;
}

In [ ]:
!g++ list_vs_vector_2.cpp -o list_vs_vector_2 && ./list_vs_vector_2


#### vectorとlistのどちらを使うべきか

「とりあえず`vector`」が基本です。<br>
`list`の利点と欠点から、`list`が役に立つのは「リストの途中への追加や削除が頻繁に行われる場合」に限られます。<br>
ですが、そういったケースは多くありません。そのため、基本的には`vector`を使うべき、となります。
